In [1]:
import numpy as np
import pygmt
from obspy.signal.filter import lowpass

def read_tgsfile(ist, dir):
    ist = str(ist).zfill(6)
    file_path = f"{dir}/tgs{ist}"
    eta = np.array([]) # do not add 0.0 as the value at t=0

    with open(file_path, 'r') as file:
        for line in file:
            parts = line.split()
            if "step=" in line:
                eta = np.append(eta, float(parts[4]))

    return eta

def lowpassfilter(eta, ts, ts_resample, resample):
    trend = (ts-ts[0])*(eta[-1]-eta[0])/(ts[-1]-ts[0]) + eta[0]
    eta_detrend = eta - trend
    f1 = 1/(ts[1]-ts[0])
    f2 = 1/(ts_resample[1]-ts_resample[0])
    lowpassed = lowpass(eta_detrend, f2/2, f1, zerophase=True) + trend
    return lowpassed[::resample]

def RelativeError(A_pred, A_true):
    return np.linalg.norm(A_pred-A_true) / np.linalg.norm(A_true)

def VarianceReduction(A_pred, A_true):
    rel_error = RelativeError(A_pred, A_true)
    return 100 * (1-rel_error**2)


In [ ]:
# config 
dir_data = "data_Tohoku"
dir_result = "result_Tohoku"

fname_station_name = "share/TohokuCoast14.txt"
scale = 15

In [5]:
# station info (coast)
with open(fname_station_name) as f:
    Stations_coast = [s.rstrip() for s in f.readlines()]

N_st = len(Stations_coast)

In [6]:
# paramters of time series
dt = 0.1               # time step for FDM calculation [sec]
dt_resample = 10       # time interval for PINN [sec]
resample = int(dt_resample/dt) 

tmin = 0 * 60
tmax = 60 * 60

Nt = int(tmax/dt) + 1
Nt_resample = int(tmax/dt_resample) + 1

ts = np.linspace(tmin, tmax, Nt)
ts_resample = ts[::resample]

In [ ]:
fig = pygmt.Figure()
with pygmt.config(FONT_LABEL="30p", FONT_ANNOT_PRIMARY="25p", MAP_TICK_PEN="2p,black", MAP_TICK_LENGTH="0.5c"):
    fig.basemap(
        region = [tmin//60, tmax//60, 0, N_st+1], 
        projection = 'X15c/35c', 
        frame = ['WSen', 'xafg+lTime [min]', 'yfg']
    )

    # arrow
    fig.plot(
        x=[2,2],y=[N_st,N_st+1],
        pen="5p,black",
        no_clip=True
    )
    fig.text(
        x = 7.5,
        y = N_st+1/2,
        font = "24p,Helvetica,black",
        text = f"{scale} m"
    )

    for i in range(N_st):
        yst = N_st - i
        ist = i+133+1
        station = Stations_coast[i]

        eta_true = np.loadtxt(dir_data+f"/tgs_filtered_coast/{station}.txt")

        eta_pred = read_tgsfile(ist, dir_result+"/pred_from_eta0/tgsfiles")
        eta_pred = lowpassfilter(eta_pred, ts, ts_resample, resample)

        VR = VarianceReduction(eta_pred, eta_true)


        if i==0:
            fig.plot(x = ts_resample/60, y = yst + eta_true / scale, pen = "3p,dimgray", label="Ground Truth")
            fig.plot(x = ts_resample/60, y = yst + eta_pred / scale, pen = "3p,red",     label="Prediction")
        else:
            fig.plot(x = ts_resample/60, y = yst + eta_true / scale, pen = "3p,dimgray")
            fig.plot(x = ts_resample/60, y = yst + eta_pred / scale, pen = "3p,red")

        fig.text(
            x = -3,
            y = yst,
            font="24p,Helvetica,black",
            justify="RM",
            text = f" {i+1}. {station} ({VR:4.1f} %)",
            no_clip=True 
        )

    fig.legend(position="JTR+jTR+o0.5c/-3.0c", box=True)
    fig.savefig(dir_result+"/coast_prediction.png")